In [7]:
# import necessary packages
import math
from random import choice
from pygmodels.pgm.pgmodel.markov import ConditionalRandomField
from pygmodels.graph.gtype.edge import Edge, EdgeType
from pygmodels.factor.factor import Factor
from pygmodels.pgm.pgmtype.randomvariable import NumCatRVariable

In [8]:
# define data and nodes
idata = {"A": {"outcome-values": [True, False]}}

# from Koller, Friedman 2009, p. 144-145, example 4.20
X_1 = NumCatRVariable(
    node_id="X_1", input_data=idata["A"], marginal_distribution=lambda x: 0.5
)
X_2 = NumCatRVariable(
    node_id="X_2", input_data=idata["A"], marginal_distribution=lambda x: 0.5
)
X_3 = NumCatRVariable(
    node_id="X_3", input_data=idata["A"], marginal_distribution=lambda x: 0.5
)
Y_1 = NumCatRVariable(
    node_id="Y_1", input_data=idata["A"], marginal_distribution=lambda x: 0.5
)

In [9]:
# define edges

X1_Y1 = Edge(
   edge_id="X1_Y1",
   edge_type=EdgeType.UNDIRECTED,
   start_node=X_1,
   end_node=Y_1,
)
X2_Y1 = Edge(
  edge_id="X2_Y1",
  edge_type=EdgeType.UNDIRECTED,
  start_node=X_2,
  end_node=Y_1,
)
X3_Y1 = Edge(
  edge_id="X3_Y1",
  edge_type=EdgeType.UNDIRECTED,
  start_node=X_3,
  end_node=Y_1,
)


In [10]:
# define factor functions

def phi_X1_Y1(scope_product):
  ""
  w = 0.5
  ss = frozenset(scope_product)
  if ss == frozenset([("X_1", True), ("Y_1", True)]):
      return math.exp(1.0 * w)
  else:
      return math.exp(0.0)
                                                                     
def phi_X2_Y1(scope_product):
  ""
  w = 5.0
  ss = frozenset(scope_product)
  if ss == frozenset([("X_2", True), ("Y_1", True)]):
      return math.exp(1.0 * w)
  else:
      return math.exp(0.0)
                                                                     
def phi_X3_Y1(scope_product):
  ""
  w = 9.4
  ss = frozenset(scope_product)
  if ss == frozenset([("X_3", True), ("Y_1", True)]):
      return math.exp(1.0 * w)
  else:
      return math.exp(0.0)
                                                                     
def phi_Y1(scope_product):
  ""
  w = 0.6
  ss = frozenset(scope_product)
  if ss == frozenset([("Y_1", True)]):
      return math.exp(1.0 * w)
  else:
      return math.exp(0.0)

In [11]:
# instantiate factors with factor functions and implied random variables
X1_Y1_f = Factor(
    gid="x1_y1_f", scope_vars=set([X_1, Y_1]), factor_fn=phi_X1_Y1
)
X2_Y1_f = Factor(
    gid="x2_y1_f", scope_vars=set([X_2, Y_1]), factor_fn=phi_X2_Y1
)
X3_Y1_f = Factor(
    gid="x3_y1_f", scope_vars=set([X_3, Y_1]), factor_fn=phi_X3_Y1
)
Y1_f = Factor(gid="y1_f", scope_vars=set([Y_1]), factor_fn=phi_Y1)

In [12]:
# Instantiate conditional random field and make a query
crf_koller = ConditionalRandomField(
    "crf",
    observed_vars=set([X_1, X_2, X_3]),
    target_vars=set([Y_1]),
    edges=set([X1_Y1, X2_Y1, X3_Y1]),
    factors=set([X1_Y1_f, X2_Y1_f, X3_Y1_f, Y1_f]),
)
evidence = set([("Y_1", False)])
query_vars = set([X_1, X_2, X_3])
query = frozenset(
    [
        ("X_1", choice([False, True])),
        ("X_2", choice([False, True])),
        ("X_3", choice([False, True])),
    ]
)
fact, a1 = crf_koller.cond_prod_by_variable_elimination(
    queries=query_vars, evidences=evidence
)
print(fact.phi(query) == 1.0)
# True



True
